In [1]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os 

load_dotenv()

access_key = os.getenv('ACCESS_KEY')
secret_key = os.getenv('SECRET_KEY')

In [2]:
spark = SparkSession.builder.appName("S3Spark")\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk:1.12.262")\
    .config("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("fs.s3a.access.key", access_key)\
    .config("fs.s3a.secret.key", secret_key)\
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
    
sqlContext = SparkSession(spark)

24/04/17 15:05:46 WARN Utils: Your hostname, Rutvik.local resolves to a loopback address: 127.0.0.1; using 10.0.0.46 instead (on interface en0)
24/04/17 15:05:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/rutvik/.ivy2/cache
The jars for the packages stored in: /Users/rutvik/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-426b0ce3-3e0d-47bd-bba5-2d515e133f83;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk;1.12.262 in central
	found com.amazonaws#aws-java-sdk-iamrolesanywhere;1.12.262 in central
	found com.amazonaws#aws-java-sdk-core;1.12.262 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.6.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.12.6 in central
	found com.fasterxml.jackson.core#jackson-core;2.12.6 in central
	found com.fasterxml.jackson.dataformat#jackson-dataformat-cbor;2.12.6 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws

In [8]:
health_camp_deets = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("s3a://ee-rs-s3-spark/Health_Camp_Detail.csv")
first_health_camp_attended = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("s3a://ee-rs-s3-spark/First_Health_Camp_Attended.csv")
second_health_camp_attended = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("s3a://ee-rs-s3-spark/Second_Health_Camp_Attended.csv")
third_health_camp_attended = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("s3a://ee-rs-s3-spark/Third_Health_Camp_Attended.csv")

In [15]:
health_camp_deets.createOrReplaceTempView("health_camp_deets")
first_health_camp_attended.createOrReplaceTempView("first_health_camp_attended")
second_health_camp_attended.createOrReplaceTempView("second_health_camp_attended")
third_health_camp_attended.createOrReplaceTempView("third_health_camp_attended")

In [16]:
spark.sql("select * from third_health_camp_attended").show(10)

+----------+--------------+-----------------------+-------------------------+
|Patient_ID|Health_Camp_ID|Number_of_stall_visited|Last_Stall_Visited_Number|
+----------+--------------+-----------------------+-------------------------+
|    517875|          6527|                      3|                        1|
|    504692|          6578|                      1|                        1|
|    504692|          6527|                      3|                        1|
|    493167|          6527|                      4|                        4|
|    510954|          6528|                      2|                        2|
|    501825|          6527|                      2|                        4|
|    495620|          6527|                      1|                        1|
|    526542|          6528|                      2|                        2|
|    517351|          6578|                      3|                        5|
|    517351|          6527|                      3|             

In [13]:
health_camp_deets.printSchema()

root
 |-- Health_Camp_ID: integer (nullable = true)
 |-- Camp_Start_Date: string (nullable = true)
 |-- Camp_End_Date: string (nullable = true)
 |-- Category1: string (nullable = true)
 |-- Category2: string (nullable = true)
 |-- Category3: integer (nullable = true)



In [14]:
health_camp_deets.select("Health_Camp_ID", "Camp_Start_date").show(10)

+--------------+---------------+
|Health_Camp_ID|Camp_Start_date|
+--------------+---------------+
|          6560|      16-Aug-03|
|          6530|      16-Aug-03|
|          6544|      03-Nov-03|
|          6585|      22-Nov-03|
|          6561|      30-Nov-03|
|          6581|      07-Dec-03|
|          6564|      17-Dec-03|
|          6557|      04-Jan-04|
|          6538|      09-Jan-04|
|          6546|      09-Jan-04|
+--------------+---------------+
only showing top 10 rows

